In [1]:
import cntk as C
from deep_model import *
import deep_model
prefix = "C:\\Users\\t-haohu\\Documents\\Python\\IndustryClassifier\\Data"
suffix = "180days_sample"

data_sample_body_ctf_train = "{}\\ready\\train_{}_body.ctf".format(prefix,suffix)
data_sample_body_ctf_test = "{}bilibi\ready\\train_{}_body.ctf".format(prefix,suffix)
data_sample_ctf_train = "{}\\ready\\train_{}.ctf".format(prefix,suffix)
data_sample_ctf_test = "{}\\ready\\test_{}.ctf".format(prefix,suffix)

data_title_dict = "{}\\ready\\title_{}.pkl".format(prefix,suffix)
data_body_dict = "{}\\ready\\body_{}.pkl".format(prefix,suffix)
data_industry_sample = "{}\\ready\\industry_{}.wl".format(prefix,suffix)

init_param_setting(vocabulary= 69516,industry = 19,title=69516,body = 388082)
#print(deep_model.vocab_size,vocab_size)
super_param_setting(embed=300,hidden=200,sentence=50)
file_param_setting(data_industry_sample,data_title_dict,data_body_dict)
models = ['cnn_2fold','cnn']
for m in models:
    do_train(m,data_sample_ctf_train,data_sample_ctf_test,suffix,48000)
models = ['cnn_2fold','cnn']
for m in models:
    do_train(m,data_sample_body_ctf_train,data_sample_body_ctf_test,suffix,48000)

cnn_2fold
(Parameter('W', [], [300 x 1 x 301 x 19]), Parameter('b', [], [19]), Parameter('W', [], [100 x 1 x 4 x 300]), Parameter('b', [], [100 x 1 x 1]), Parameter('E', [], [69516 x 300]), Parameter('W', [], [100 x 1 x 3 x 300]), Parameter('b', [], [100 x 1 x 1]), Parameter('W', [], [100 x 1 x 5 x 300]), Parameter('b', [], [100 x 1 x 1]))
Training 22930819 parameters in 9 parameter tensors.


RuntimeError: CUDA failure 4: unspecified launch failure ; GPU=0 ; hostname=MININT-8LA6JR2 ; expr=cudaDeviceSynchronize()

[CALL STACK]
    > Microsoft::MSR::CNTK::CudaTimer::  Stop
    - Microsoft::MSR::CNTK::CudaTimer::  Stop (x2)
    - RtlRunOnceExecuteOnce
    - InitOnceExecuteOnce
    - _crtInitOnceExecuteOnce
    - Microsoft::MSR::CNTK::GPUMatrix<half>::  GetCublasHandle
    - Microsoft::MSR::CNTK::GPUSparseMatrix<half>::  performElementWiseFunction (x2)
    - Microsoft::MSR::CNTK::GPUMatrix<float>::  TensorOp
    - Microsoft::MSR::CNTK::Matrix<float>::  TensorOp
    - Microsoft::MSR::CNTK::TensorView<float>::  DoUnaryOpOf
    - Microsoft::MSR::CNTK::TensorView<float>::  AssignCopyOf
    - std::enable_shared_from_this<Microsoft::MSR::CNTK::MatrixBase>::  shared_from_this (x2)
    - CNTK::Internal::  UseSparseGradientAggregationInDataParallelSGD



In [7]:
import cntk as C
import numpy as np
import cntk.tests.test_utils
import pickle
from cntk import sequence
from cntk import load_model

cntk.tests.test_utils.set_device_from_pytest_env() # (only needed for our build system)
C.cntk_py.set_fixed_random_seed(1) # fix a random seed for CNTK components


from cntk.device import try_set_default_device, gpu,cpu
try_set_default_device(cpu())
vocab_size = 23184
num_labels = 19
title_size = 23184
body_size = 210000
# model dimensions
input_dim  = vocab_size
label_dim  = num_labels
emb_dim    = 300
hidden_dim = 200
max_length_title = 50
max_length = 50
max_length_body = 200
model = 'cnn'
minibatch_size = 8
prefix = "C:\\Users\\t-haohu\\Documents\\Python\\IndustryClassifier\\Data"
suffix = "180days_sample"

def create_reader(path, is_training,is_body=False):
    if is_body:
        return C.io.MinibatchSource(C.io.CTFDeserializer(path, C.io.StreamDefs(
         title         = C.io.StreamDef(field='S0', shape=title_size,  is_sparse=True),
         industry        = C.io.StreamDef(field='S1', shape=num_labels, is_sparse=True),
         body        = C.io.StreamDef(field='S2', shape=body_size, is_sparse=True),
     )), randomize=is_training, max_sweeps = C.io.INFINITELY_REPEAT if is_training else 1)
    else:
        return C.io.MinibatchSource(C.io.CTFDeserializer(path, C.io.StreamDefs(
         title         = C.io.StreamDef(field='S0', shape=vocab_size,  is_sparse=True),
         industry        = C.io.StreamDef(field='S1', shape=num_labels, is_sparse=True),
         
     )), randomize=is_training, max_sweeps = C.io.INFINITELY_REPEAT if is_training else 1)



def create_criterion_function(model):
    labels = C.placeholder(name='labels')
    ce   = C.cross_entropy_with_softmax(model, labels)
    errs = C.classification_error      (model, labels)
    return C.combine ([ce, errs]) # (features, labels) -> (loss, metric)

def create_criterion_function_preferred(model, labels):
    ce   = C.cross_entropy_with_softmax(model, labels)
    errs = C.classification_error      (model, labels,topN=1)
    return ce, errs # (model, labels) -> (loss, error metric)

embed_title_file = "{}\\ready\\title_{}.pkl".format(prefix,suffix)
embed_body_file = "{}\\ready\\body_{}.pkl".format(prefix,suffix)
industry_file = "{}\\ready\\industry_{}.wl".format(prefix,suffix)
industry=[i.rstrip("\n") for i in open(industry_file,encoding = "utf-8").readlines()]
if embed_title_file:
    with open(embed_title_file,'rb') as handle:
        embedding_title=pickle.load(handle)
if embed_body_file:
    with open(embed_body_file,'rb') as handle:
        embedding_body=pickle.load(handle)

reader = create_reader("Data//ready//train_180days_sample.ctf", is_training=True)
x = C.sequence.input_variable(vocab_size)
y = C.input_variable(num_labels) 


input_dim=hidden_dim
output_dim = hidden_dim
#data_map={x: reader.streams.title, y: reader.streams.industry}
data = reader.next_minibatch(20)
for key in data.keys():
        if(key.m_name=="title"):

            test_x=data[key]

        if(key.m_name=="industry"):

            test_y=data[key]
print(test_x,test_x.shape)


h1_1= C.layers.Embedding(weights=embedding_title,name='embed')(x)#
#h1_1= C.layers.Embedding(300,name='embed')(x)
h1_2= C.layers.Embedding(300,name='embed')(x)#init=embedding,


#bn = C.layers.BatchNormalization(name='bn')(h1)
to_static_1= C.layers.PastValueWindow(window_size=max_length_title, axis=-2)(h1_1)[0]
to_static_2= C.layers.PastValueWindow(window_size=max_length_title, axis=-2)(h1_2)[0]
#value,valid = to_static(h1)
h1_1_expand = C.expand_dims(to_static_1, -3)
h1_2_expand = C.expand_dims(to_static_2, -3)
h1 = C.splice(h1_1_expand,h1_2_expand,axis = -3)
filter_num=100
emb_dim =300
h2_1=C.layers.Convolution((3,emb_dim),num_filters=filter_num,reduction_rank=1,activation=C.relu)(h1)
#h2_2=C.layers.Convolution((4,emb_dim),num_filters=filter_num,reduction_rank=1,activation=C.relu)(to_static)
#h2_3=C.layers.Convolution((5,emb_dim),num_filters=filter_num,reduction_rank=1,activation=C.relu)(to_static)

#h3_1=C.layers.MaxPooling((max_length-2,1),name='pooling')(h2_1)
#h3_2=C.layers.MaxPooling((max_length-3,1),name='pooling')(h2_2)
#h3_3=C.layers.MaxPooling((max_length-4,1),name='pooling')(h2_3)

#h3=C.splice(h3_2,h3_1,h3_3,axis=0)
#drop1 =C.layers.Dropout(0.5)(h3)
#h4=C.layers.Dense(num_labels,name='hidden')(drop1)

print(np.array(h1_1.eval({x:test_x})).shape)
print(np.array(h1_2.eval({x:test_x})).shape)
print(to_static_1)
print(to_static_2)
print(np.array(h1_1_expand.eval({x:test_x})).shape)
print(np.array(h1_2_expand.eval({x:test_x})).shape)
print(np.array(h1.eval({x:test_x})).shape)


print(np.array(h2_1.eval({x:test_x})).shape)
#print(np.array(h2_3.eval({x:test_x})).shape)
#print(np.array(conv1_2.eval({x:test_x})).shape)
print(np.array(h3_2.eval({x:test_x})).shape)
#print(np.array(h3_3.eval({x:test_x})).shape)
print("here")
#print(np.array(h3.eval({x:test_x})).shape)
print(h4.parameters)
print(np.array(h4.eval({x:test_x})).shape)
#print(np.array(h5.eval({x:test_x})).shape)
#print(np.array(h3.eval({x:test_x})).shape)

MinibatchData(data=Value([1 x 22 x 23184], CPU), samples=22, seqs=1) (1, 22, 23184)
(1, 22, 300)
(1, 22, 300)
Output('Block253189_Output_0', [#], [50 x 300])
Output('Block270741_Output_0', [#], [50 x 300])
(1, 1, 50, 300)
(1, 1, 50, 300)
(1, 2, 50, 300)
(1, 100, 48, 1)


NameError: name 'h3_2' is not defined